In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests
import time
import json
import pandas as pd

In [3]:
pages = [1,2]

#creating dataframe to store the job opening details

column_names = ["Job Title","Company Name","Company Domain","Location","Skills","Employement Type","Experience Level"]
df = pd.DataFrame(columns = column_names)

for page in pages:
    source = requests.get("https://stackoverflow.com/jobs?pg={}".format(page)).text
    soup = BeautifulSoup(source,'lxml')
    jobs = soup.find(class_='listResults')
    for job in jobs.find_all(class_='-job'): 
        
        #accessing the job description url for each job from <a> tag
        
        link = job.h2.find('a',class_='stretched-link')['href']
        job_content = requests.get("https://stackoverflow.com{}".format(link)).text
        bs = BeautifulSoup(job_content,'lxml')
        
        #accessing json file from script tag
        
        script_tag = bs.find("script",{"type":"application/ld+json"})
        if script_tag.contents[0]:
            json_listing_contents = json.loads(script_tag.contents[0])
            
            #code to fetch job opening details like job title , company name , address etc from json
            
            if(len(json_listing_contents["title"]) > 0):
                job_title = json_listing_contents["title"]
            else:
                job_title = None
                
            if(len(json_listing_contents["hiringOrganization"]["name"]) > 0):
                company_name = json_listing_contents["hiringOrganization"]["name"]
            else:
                company_name = None
                
            if('skills' in json_listing_contents):
                if(len(json_listing_contents["skills"]) > 0):
                    skills = ""
                    for skill in json_listing_contents["skills"]:
                        skills += skill+","
                else:
                    skills = None
            else:
                skills = None
                
            if('industry' in json_listing_contents):
                if(len(json_listing_contents["industry"]) > 0):
                    company_domain = ""
                    for domain in json_listing_contents["industry"]:
                        company_domain += domain+","
                else:
                    company_domain = None
            else:
                company_domain = None
                
            if('experienceRequirements' in json_listing_contents):
                experienceRequirements = json_listing_contents["experienceRequirements"]
            else:
                experienceRequirements = None
                
            if('employmentType' in json_listing_contents):
                employmentType = json_listing_contents["employmentType"]
            else:
                employmentType = None
                
            if('address' in json_listing_contents["jobLocation"][0]):
                location = ""
                for (k, v) in json_listing_contents["jobLocation"][0] ["address"].items():
                    if(str(v)=="PostalAddress"):
                        location = ""
                    elif(str(v) == "-"):
                        location += ""
                    else:
                        location += str(v)+","
            else:
                location = None

            df = df.append({'Job Title':job_title,'Company Name':company_name,'Company Domain':company_domain,'Location':location,'Skills':skills,'Employement Type':employmentType,'Experience Level':experienceRequirements}, ignore_index=True)
    time.sleep(2)
df

,Job Title,Company Name,Company Domain,Location,Skills,Employement Type,Experience Level
0,"Senior Full-Stack Software Engineer, OS Perfor...",Apple,"Consumer Electronics,","US,CA,San Diego,","linux,",FULL_TIME,None
1,【フルスタックエンジニア(大規模Webサービス)】株式会社ビットエー<五反田駅徒歩5分×残業少>,株式会社ヴィジョナリー,None,"JP,Shinagawa City,","laravel,django,javascript,",FULL_TIME,None
2,"PHP Developer, Software Engineer",Zeki Software Solutions,None,"IN,Mumbai,","php,mysql,jquery,",FULL_TIME,None
3,Mobile Software App developer,Knish Technologies,None,"IN,Mumbai,","javascript,reactjs,git,",FULL_TIME,None
4,Full Stack Software Engineer,Palantir,"Computer Software,Data & Analytics,","US,WA,Seattle,","reactjs,javascript,java,",FULL_TIME,None
5,Full Stack Software Engineer,Palantir,"Computer Software,Data & Analytics,","US,NY,New York,","reactjs,javascript,java,",FULL_TIME,None
6,PHP Web Developer,WECRAFT SOLUTIONS LLP,None,"IN,Navi Mumbai,","php,javascript,jquery,",FULL_TIME,None
7,Freelance_Angular JS,2COMs,None,"IN,Mumbai,","angularjs,jquery,javascript,",FULL_TIME,None
8,Backend Developer,PUBBURPS,None,"IN,Bengaluru,","mongodb,amazon-web-services,clojure,",FULL_TIME,None
9,Lead/Senior Backend Developer - We're motivate...,Kind,"Healthcare,","SE,","microservices,kubernetes,spring-boot,java,java...",FULL_TIME,"Senior, Lead"


In [5]:
#storing job opening details in excel
df.to_excel('Job_Openings.xlsx', index=False) 